<a href="https://www.kaggle.com/code/minhbtnguyen/wine-sales-prediction-with-multi-layer-perceptron?scriptVersionId=126839896" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Wine Sales Prediction Playground

- Date: 5/12/2022

*Pls upvote if you found this notebook helpful*

- Community Prediction Competition - Binary Classification.
- [Link](https://www.kaggle.com/competitions/predict-wine-sales-2022/data?select=logit_wine_train.csv)

## 1. Import Packages

**P/S**: if you are new to these package, please either install them via terminal using [Anaconda](https://docs.anaconda.com/anaconda/user-guide/tasks/install-packages/) or [pip](https://packaging.python.org/en/latest/tutorials/installing-packages/). Just google the name of the package with either "conda install ABC" or "pip install ABC".

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

## 2. Import Data
Import both training and testing datasets for analysis and feature selections. From the features' distributions of the [dataset](https://www.kaggle.com/competitions/predict-wine-sales-2022/data?select=logit_wine_test.csv), we can see that all features' distributions of logit_wine_train.csv are similar to their corresponding features' in logit_wine_test.csv. This is very helpful since our trained model can be generalized in towards the test dataset. It seems like the testing dataset is randomly withdrawn from the trainig set at (80%,20%) train-test-split: (12794, 3334) respectively. 

In [2]:
# Import training dataset.
raw_train_df = pd.read_csv("../input/predict-wine-sales-2022/logit_wine_train.csv")
raw_train_df.head()

,INDEX,TARGET,FixedAcidity,VolatileAcidity,CitricAcid,ResidualSugar,Chlorides,FreeSulfurDioxide,TotalSulfurDioxide,Density,pH,Sulphates,Alcohol,LabelAppeal,AcidIndex,STARS
0,1,1,3.2,1.160,-0.98,54.2,-0.567,NaN,268.0,0.99280,3.33,-0.59,9.9,0,8,2.0
1,2,1,4.5,0.160,-0.81,26.1,-0.425,15.0,-327.0,1.02792,3.38,0.70,NaN,-1,7,3.0
2,4,1,7.1,2.640,-0.88,14.8,0.037,214.0,142.0,0.99518,3.12,0.48,22.0,-1,8,3.0
3,5,1,5.7,0.385,0.04,18.8,-0.425,22.0,115.0,0.99640,2.24,1.83,6.2,-1,6,1.0
4,6,1,8.0,0.330,-1.26,9.4,NaN,-167.0,108.0,0.99457,3.12,1.77,13.7,0,9,2.0


In [3]:
# Import testing dataset.
raw_test_df = pd.read_csv("../input/predict-wine-sales-2022/logit_wine_test.csv")
raw_test_df.head()

,INDEX,TARGET,FixedAcidity,VolatileAcidity,CitricAcid,ResidualSugar,Chlorides,FreeSulfurDioxide,TotalSulfurDioxide,Density,pH,Sulphates,Alcohol,LabelAppeal,AcidIndex,STARS
0,3,NaN,5.4,-0.860,0.27,-10.7,0.092,23.0,398.0,0.98527,5.02,0.64,12.30,-1,6,NaN
1,9,NaN,12.4,0.385,-0.76,-19.7,1.169,-37.0,68.0,0.99048,3.37,1.09,16.00,0,6,2.0
2,10,NaN,7.2,1.750,0.17,-33.0,0.065,9.0,76.0,1.04641,4.61,0.68,8.55,0,8,1.0
3,18,NaN,6.2,0.100,1.80,1.0,-0.179,104.0,89.0,0.98877,3.20,2.11,12.30,-1,8,1.0
4,21,NaN,11.4,0.210,0.28,1.2,0.038,70.0,53.0,1.02899,2.54,-0.07,4.80,0,10,NaN


At very first glance at this dataset, I saw the word "volitile", I immediately thought of stock dataset, similar to [Google Stock Price Dataset](https://www.kaggle.com/datasets/medharawat/google-stock-price). Ok, since the dataset has "index", no description, and no time-stamp, I do not think this dataset is time-series dataset, this is a tabular dataset. We will ignore INDEX, predict "TARGET", and use the rest of features to train model.

### Feature Understanding (J4F)
- [Source](https://rstudio-pubs-static.s3.amazonaws.com/57835_c4ace81da9dc45438ad0c286bcbb4224.html). This site also has way more in-depth data analysis on the dataset!

I am no wine expert or chemist. However, I think understanding the dataset is equally or more important than model building process since we can do some fancy preprocessing (if neccessary) and even select the right model.
- FixedAcidity: most acids involved with wine or fixed or nonvolatile (do not evaporate readily).
- VolatileAcidity: the amount of acetic acid in wine, which at too high of levels can lead to an unpleasant, vinegar taste.
- CitricAcid: found in small quantities, citric acid can add ‘freshness’ and flavor to wines.
- ResidualSugar: the amount of sugar remaining after fermentation stops, it’s rare to find wines with less than 1 gram/liter and wines with greater than 45 grams/liter are considered sweet.
- Chlorides: the amount of salt in the wine.
- FreeSulfurDioxide: the free form of SO2 exists in equilibrium between molecular SO2 (as a dissolved gas) and bisulfite ion; it prevents microbial growth and the oxidation of wine.
- TotalSulfureDioxide: amount of free and bound forms of S02; in low concentrations, SO2 is mostly undetectable in wine, but at free SO2 concentrations over 50 ppm, SO2 becomes evident in the nose and taste of wine. 
- Density: the density of water is close to that of water depending on the percent alcohol and sugar content
- pH: describes how acidic or basic a wine is on a scale from 0 (very acidic) to 14 (very basic); most wines are between 3-4 on the pH scale.
- Sulphates: a wine additive which can contribute to sulfur dioxide gas (S02) levels, wich acts as an antimicrobial and antioxidant.
- Alcohol: the percent alcohol content of the wine.

In [4]:
# Skew-ness analysis
target_train = raw_train_df.TARGET
target_train.value_counts()

1    10061
0     2734
Name: TARGET, dtype: int64

We can see that this is an imbalance dataset, thus Accuracy will not be a good metric for validation set. We will use [F1-score](https://en.wikipedia.org/wiki/F-score) since it accounts both true positive and false positive

## 3. Data Preprocessing
There are some missing data, let's do data imputation and feature normalization.

### Training Dataset
Get rid of index, data imputation, feature normalization, train-validation split.

In [5]:
# Drop INDEX column
processed_train_df = raw_train_df.drop(columns=['INDEX'])

# Data Imputation with means of each feature
features_mean = processed_train_df.mean()
processed_train_df = processed_train_df.fillna(value=features_mean)

# Min-max Normalization
# Source: https://stackoverflow.com/questions/26414913/normalize-columns-of-pandas-data-frame
normalized_train_df = (processed_train_df-processed_train_df.min())/(processed_train_df.max()-processed_train_df.min())

# Features selection
y_train_full = normalized_train_df.TARGET
X_train_full = normalized_train_df[['FixedAcidity', 
     'VolatileAcidity', 
     'CitricAcid', 
     'ResidualSugar', 
     'Chlorides', 
     'FreeSulfurDioxide', 
     'TotalSulfurDioxide',
     'Density',
     'pH',
     'Sulphates', 
     'Alcohol', 
     'LabelAppeal',
     'AcidIndex',
     'STARS']]

# Train-validation split
train_X, val_X, train_y, val_y = train_test_split(X_train_full, y_train_full, random_state=1)

### Testing Dataset
Save index, data imputation, feature normalization, train-validation split.

In [6]:
# Save index since we will submit INDEX with TARGET later
submit_df = raw_test_df.INDEX

# Drop INDEX and TARGET columns
processed_test_df = raw_test_df.drop(columns=['INDEX', 'TARGET'])

# Data Imputation with means of each feature
features_mean = processed_test_df.mean()
processed_test_df = processed_test_df.fillna(value=features_mean)

# Min-max Normalization
# Source: https://stackoverflow.com/questions/26414913/normalize-columns-of-pandas-data-frame
normalized_test_df = (processed_test_df-processed_test_df.min())/(processed_test_df.max()-processed_test_df.min())

# Features selection
X_test_full = normalized_test_df[['FixedAcidity', 
     'VolatileAcidity', 
     'CitricAcid', 
     'ResidualSugar', 
     'Chlorides', 
     'FreeSulfurDioxide', 
     'TotalSulfurDioxide',
     'Density',
     'pH',
     'Sulphates', 
     'Alcohol', 
     'LabelAppeal',
     'AcidIndex',
     'STARS']]

## 4. SVM Classifier Trial


In [7]:
# Create model
svm_classifier = SVC(random_state=1)

# Train model 
svm_classifier.fit(train_X, train_y)

# Get validation prediction
svm_classifier_val_predictions = svm_classifier.predict(val_X)

# Get validation metrics
svm_val_mae = mean_absolute_error(val_y, svm_classifier_val_predictions)
svm_val_mse = mean_squared_error(val_y, svm_classifier_val_predictions)
svm_val_accuracy = accuracy_score(val_y, svm_classifier_val_predictions)
svm_val_f1 = f1_score(val_y, svm_classifier_val_predictions, average='binary')

print(f"Validation MAE of SVM is: {svm_val_mae}")
print(f"Validation MSE of SVM is: {svm_val_mse}")
print(f"Validation Accuracy of SVM is: {svm_val_accuracy}")
print(f"Validation F1-SCore of SVM is: {svm_val_f1}")

Validation MAE of SVM is: 0.1997499218505783
Validation MSE of SVM is: 0.1997499218505783
Validation Accuracy of SVM is: 0.8002500781494217
Validation F1-SCore of SVM is: 0.8860353130016052


## 5. MLP Classifier Trial



In [8]:
# Create model
mlp_classifier = MLPClassifier(random_state=1, max_iter=1000, hidden_layer_sizes=(100,100,50,25,10))
# mlp_classifier = MLPClassifier(random_state=1, max_iter=5000, hidden_layer_sizes=(200,200,100,50,25,10))

# Train model 
mlp_classifier.fit(train_X, train_y)

# Get validation prediction
mlp_classifier_val_predictions = mlp_classifier.predict(val_X)

# Get validation metrics
mlp_val_mae = mean_absolute_error(val_y, mlp_classifier_val_predictions)
mlp_val_mse = mean_squared_error(val_y, mlp_classifier_val_predictions)
mlp_val_accuracy = accuracy_score(val_y, mlp_classifier_val_predictions)
mlp_val_f1 = f1_score(val_y, mlp_classifier_val_predictions, average='binary')

print(f"Validation MAE of SVM is: {mlp_val_mae}")
print(f"Validation MSE of SVM is: {mlp_val_mse}")
print(f"Validation Accuracy of SVM is: {mlp_val_accuracy}")
print(f"Validation F1-SCore of SVM is: {mlp_val_f1}")

Validation MAE of SVM is: 0.1491090965926852
Validation MSE of SVM is: 0.1491090965926852
Validation Accuracy of SVM is: 0.8508909034073148
Validation F1-SCore of SVM is: 0.9044279703466238


## 6. MLP Classifier Trained on Full Dataset
We can see that MLP is better than SVM, thus we will choose MLP for full trainning dataset.

In [9]:
# Create model
mlp_classifier_full = MLPClassifier(random_state=1, max_iter=5000, hidden_layer_sizes=(100,100,50,25,10))

# Train model 
mlp_classifier_full.fit(X_train_full, y_train_full)

MLPClassifier(hidden_layer_sizes=(100, 100, 50, 25, 10), max_iter=5000,
              random_state=1)

In [10]:
test_preds = mlp_classifier_full.predict(X_test_full)

## 7. Generate Submission

Generate submission wine_prediction_submission.csv.

In [11]:
output = pd.DataFrame({'INDEX': raw_test_df.INDEX,
                       'P_TARGET': test_preds})
output.to_csv('submission.csv', index=False)